# <center><font color='red'>5 - PROCESSING OF INDIVIDUAL CELLS</font></center>

### <center><font color='orange'>AUTHOR: Diogo Vieira</font></center>

-------------------------------------------

### <center><font color='grey'>Imports</font></center>

In [1]:
# Math, image processing and other useful libraries
from __future__ import print_function, unicode_literals, absolute_import, division
import os

import pandas as pd
import numpy as np
import cv2
from collections import OrderedDict
import copy
import math
import pickle
from matplotlib.ticker import MaxNLocator
from itertools import combinations

# Image processing
from skimage.measure import regionprops
from skimage.filters import meijering, sato, frangi, hessian, threshold_otsu
from skimage.morphology import extrema, skeletonize
from skimage.transform import probabilistic_hough_line
from skimage.draw import disk, circle_perimeter
from scipy.ndimage import gaussian_filter, grey_closing
from scipy.spatial import distance_matrix
from skimage import data, restoration, util
from roipoly import RoiPoly
from matplotlib_scalebar.scalebar import ScaleBar
from biosppy.signals import tools
from biosppy.stats import pearson_correlation
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Plotting
import matplotlib.pyplot as plt
import matplotlib.cm as pltc
import matplotlib.colors as colors
import seaborn as sns

# Widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display

# Feature Extraction (.py files by Teresa Parreira)
# from CytoSkeletonPropsMorph import CytoSkeletonPropsMorph
# from CytoSkeletonRegionPropsInt import RegionPropsInt
# from FreqAnalysis import FreqAnalysis
# from GLCM import GLCM

# Graph
import sknw
import networkx as nx
from scipy.signal import argrelextrema

# 
from skan import Skeleton, summarize,draw
from skan.csr import skeleton_to_csgraph, sholl_analysis,make_degree_image
import scipy as sp
import scipy.sparse
from matplotlib.patches import Circle
from framework.ImageFeatures import ImageFeatures,getvoxelsize
from framework.Functions import cv2toski,pylsdtoski,polar_to_cartesian, remove_not1D, quantitative_analysis,hist_bin,hist_lim,create_separate_DFs,branch,graphAnalysis
from framework.Importing import label_image,init_import
from framework.PreProcessingCYTO import cytoskeleton_preprocessing, df_cytoskeleton_preprocessing
from framework.PreProcessingNUCL import excludeborder, nuclei_preprocessing, df_nuclei_preprocessing, nuclei_segmentation
from framework.Processing import process3Dnuclei,analyze_cell,df_analyze_cell
from framework.visualization import truncate_colormap, plot_hist, plot_pie
#from fractal_dimension import fractal_dimension
#from fractal_analysis_fxns import boxcount,boxcount_grayscale,fractal_dimension,fractal_dimension_grayscale,fractal_dimension_grayscale_DBC

print('📚 All libraries successfully imported 📚')

📚 All libraries successfully imported 📚


#### <center><font color='grey'>Directories</font></center>

In [2]:
### DATASET #1 (2D)
folder    = os.path.dirname(os.getcwd()) + "\\Datasets\\Set 1-a-tubulin_Sofia"
options   = ["RGB","CYTO_DECONV","NUCL_DECONV"]

### DATASET #2 (3D)
#folder    = os.path.dirname(os.getcwd()) + "\\Datasets\\Set 3D"
#options   = ["3D"]

### DATASET #3 (Soraia)
# folder    = os.path.dirname(os.getcwd()) + "\\Datasets\\Soraia"
# options   = ["3D"]

### DATASET #4 (Inês)
# folder    = os.path.dirname(os.getcwd()) + "\\Datasets\\Ines"
#options     = []


data = init_import(folder,options,label_image)
data['NUCL_PRE'] = pd.read_pickle(folder + "\\NUCL_PRE\\NUCL_PRE.pkl") # pd.read_pickle(os.getcwd() + "\\NUCL_PRE_2D_StarDist.pkl") #
data['CYTO_PRE'] = pd.read_pickle(folder + "\\CYTO_PRE\\CYTO_PRE.pkl") 

>>> [RGB] added.
>>> [CYTO_DECONV] added.
>>> [NUCL_DECONV] added.


In [1]:
import ipywidgets as widgets

# Define the groups of features
groups = {
    'DCF': [
        'DCF:Area',
        'DCF:BB Area',
        'DCF:Perimeter',
        'DCF:Area convex',
        'DCF:Centroid',
        'DCF:Weighted Centroid',
        'DCF:Centroid Divergence',
        'DCF:Equivalent Diameter',
        'DCF:Extent',
        'DCF:Major Axis Length',
        'DCF:Minor Axis Length',
        'DCF:Height',
        'DCF:Euler Number',
        'DCF:Eccentricity',
        'DCF:Circularity',
        'DCF:Roundness',
        'DCF:Orientation',
        'DCF:Solidity',
        'DCF:Roughness',
        'DCF:Hu Moment #1',
        'DCF:Hu Moment #2',
        'DCF:Hu Moment #3',
        'DCF:Weighted Hu Moment #1',
        'DCF:Weighted Hu Moment #1',
        'DCF:Weighted Hu Moment #1',
        'DCF:Feret Diameter Max',
        'DCF:Crofton Perimeter',

        'DCF:Mean Intensity',
        'DCF:Std',
        'DCF:Variance',
        'DCF:Skewness',
        'DCF:Kurtosis',
        'DCF:Contrast',
        'DCF:Max Intensity',
        'DCF:Min Intensity',
        'DCF:Entropy',
        'DCF:Inertia Tensor Highest Eigenvalue',
        'DCF:Inertia Tensor Lowest Eigenvalue',

        'DCF:Uniformity',
        'DCF:Invariant Uniformity',
        'DCF:GLCM Entropy',
        'DCF:GLCM Invariant Entropy',
        'DCF:Correlation',
        'DCF:Invariant Correlation',
        'DCF:Dissimilarity',
        'DCF:Invariant Dissimilarity',
        'DCF:Contrast',
        'DCF:Invariant Contrast',
        'DCF:Homogeneity',
        'DCF:Invariant Homogeneity',
        'DCF:Energy',
        'DCF:Invariant Energy',

        'DCF:Mean Gabor Power',
        'DCF:Gabor Variance',
        'DCF:Gabor Mean',
        'DCF:Gabor Energy',
        'DCF:Gabor Entropy',
        'DCF:Mean Spectral Magnitude',
        'DCF:Mean Spectral Power'
    ],
    'DNF': [
            'DNF:Area',
            'DNF:BB Area',
            'DNF:Perimeter',
            'DNF:Area convex',
            'DNF:Centroid',
            'DNF:Weighted Centroid',
            'DNF:Centroid Divergence',
            'DNF:Equivalent Diameter',
            'DNF:Extent',
            'DNF:Major Axis Length',
            'DNF:Minor Axis Length',
            'DNF:Height',
            'DNF:Euler Number',
            'DNF:Eccentricity',
            'DNF:Circularity',
            'DNF:Roundness',
            'DNF:Orientation',
            'DNF:Solidity',
            'DNF:Roughness',
            'DNF:Hu Moment #1',
            'DNF:Hu Moment #2',
            'DNF:Hu Moment #3',
            'DNF:Weighted Hu Moment #1',
            'DNF:Weighted Hu Moment #1',
            'DNF:Weighted Hu Moment #1',
            'DNF:Feret Diameter Max',
            'DNF:Crofton Perimeter',
            
            'DNF:Mean Intensity',
            'DNF:Std',
            'DNF:Variance',
            'DNF:Skewness',
            'DNF:Kurtosis',
            'DNF:Contrast',
            'DNF:Max Intensity',
            'DNF:Min Intensity',
            'DNF:Entropy',
            'DNF:Inertia Tensor Highest Eigenvalue',
            'DNF:Inertia Tensor Lowest Eigenvalue',
            
            'DNF:Uniformity',
            'DNF:Invariant Uniformity',
            'DNF:GLCM Entropy',
            'DNF:GLCM Invariant Entropy',
            'DNF:Correlation',
            'DNF:Invariant Correlation',
            'DNF:Dissimilarity',
            'DNF:Invariant Dissimilarity',
            'DNF:Contrast',
            'DNF:Invariant Contrast',
            'DNF:Homogeneity',
            'DNF:Invariant Homogeneity',
            'DNF:Energy',
            'DNF:Invariant Energy',
            
            'DNF:Mean Gabor Power',
            'DNF:Gabor Variance',
            'DNF:Gabor Mean',
            'DNF:Gabor Energy',
            'DNF:Gabor Entropy',
            'DNF:Mean Spectral Magnitude',
            'DNF:Mean Spectral Power'
    ],
    'LSF': [
            'LSF2D:Angles',
            'LSF2D:Distances to Centroid',
            'LSF2D:Triangle Areas',
            'LSF2D:Line Lengths',
            'LSF2D:Theta',
            'LSF2D:Angle Difference',
            'LSF2D:Std. Angle Difference',
            'LSF2D:Local Line Distance',
            'LSF2D:Std. Local Line Distance',
            'LSF2D:PAD',
           
            'LSF1D:Number of Lines',
            'LSF1D:Radial Score',
            'LSF1D:Complete Diameter Distance',
            'LSF1D:Average Diameter Distance',
            'LSF1D:TAD',
            'LSF1D:OOP',
            'LSF1D:HI',
            'LSF1D:MCM'
    ],
    'CNF': [
            'CNF1D:Number of Branches',

            'CNF2D:branch-distance',
            'CNF1D:branch-distance mean',
            'CNF1D:branch-distance median',
            'CNF1D:branch-distance min',
            'CNF1D:branch-distance max',
            'CNF1D:branch-distance max_amp',
            'CNF1D:branch-distance var',
            'CNF1D:branch-distance std_dev',
            'CNF1D:branch-distance abs_dev',
            'CNF1D:branch-distance kurtosis',
            'CNF1D:branch-distance skewness',
            
            'CNF2D:mean-pixel-value',
            'CNF1D:mean-pixel-value mean',
            'CNF1D:mean-pixel-value median',
            'CNF1D:mean-pixel-value min',
            'CNF1D:mean-pixel-value max',
            'CNF1D:mean-pixel-value max_amp',
            'CNF1D:mean-pixel-value var',
            'CNF1D:mean-pixel-value std_dev',
            'CNF1D:mean-pixel-value abs_dev',
            'CNF1D:mean-pixel-value kurtosis',
            'CNF1D:mean-pixel-value skewness',
            
            'CNF2D:stdev-pixel-value',
            'CNF1D:stdev-pixel-value mean',
            'CNF1D:stdev-pixel-value median',
            'CNF1D:stdev-pixel-value min',
            'CNF1D:stdev-pixel-value max',
            'CNF1D:stdev-pixel-value max_amp',
            'CNF1D:stdev-pixel-value var',
            'CNF1D:stdev-pixel-value std_dev',
            'CNF1D:stdev-pixel-value abs_dev',
            'CNF1D:stdev-pixel-value kurtosis',
            'CNF1D:stdev-pixel-value skewness',
            
            'CNF2D:euclidean-distance',
            'CNF1D:euclidean-distance mean',
            'CNF1D:euclidean-distance median',
            'CNF1D:euclidean-distance min',
            'CNF1D:euclidean-distance max',
            'CNF1D:euclidean-distance max_amp',
            'CNF1D:euclidean-distance var',
            'CNF1D:euclidean-distance std_dev',
            'CNF1D:euclidean-distance abs_dev',
            'CNF1D:euclidean-distance kurtosis',
            'CNF1D:euclidean-distance skewness',
            
            'CNF1D:Number of Endpoint-to-endpoint (isolated branch)',
            'CNF1D:Ratio of Endpoint-to-endpoint (isolated branch)',
            'CNF1D:Mean of Endpoint-to-endpoint (isolated branch) branch-distance',
            'CNF1D:Std of Endpoint-to-endpoint (isolated branch) branch-distance',
            'CNF1D:Mean of Endpoint-to-endpoint (isolated branch) mean-pixel-value',
            'CNF1D:Std of Endpoint-to-endpoint (isolated branch) mean-pixel-value',
            'CNF1D:Mean of Endpoint-to-endpoint (isolated branch) stdev-pixel-value',
            'CNF1D:Std of Endpoint-to-endpoint (isolated branch) stdev-pixel-value',
            'CNF1D:Mean of Endpoint-to-endpoint (isolated branch) euclidean-distance',
            'CNF1D:Std of Endpoint-to-endpoint (isolated branch) euclidean-distance',
            
            'CNF1D:Number of Junction-to-endpoints',
            'CNF1D:Ratio of Junction-to-endpoints',
            'CNF1D:Mean of Junction-to-endpoints branch-distance',
            'CNF1D:Std of Junction-to-endpoints branch-distance',
            'CNF1D:Mean of Junction-to-endpoints mean-pixel-value',
            'CNF1D:Std of Junction-to-endpoints mean-pixel-value',
            'CNF1D:Mean of Junction-to-endpoints stdev-pixel-value',
            'CNF1D:Std of Junction-to-endpoints stdev-pixel-value',
            'CNF1D:Mean of Junction-to-endpoints euclidean-distance',
            'CNF1D:Std of Junction-to-endpoints euclidean-distance',
            
            'CNF1D:Number of Junction-to-junctions',
            'CNF1D:Ratio of Junction-to-junctions',
            'CNF1D:Mean of Junction-to-junctions branch-distance',
            'CNF1D:Std of Junction-to-junctions branch-distance',
            'CNF1D:Mean of Junction-to-junctions mean-pixel-value',
            'CNF1D:Std of Junction-to-junctions mean-pixel-value',
            'CNF1D:Mean of Junction-to-junctions stdev-pixel-value',
            'CNF1D:Std of Junction-to-junctions stdev-pixel-value',
            'CNF1D:Mean of Junction-to-junctions euclidean-distance',
            'CNF1D:Std of Junction-to-junctions euclidean-distance',
            
            'CNF1D:Number of Isolated cycles',
            'CNF1D:Ratio of Isolated cycles',
            'CNF1D:Mean of Isolated cycles branch-distance',
            'CNF1D:Std of Isolated cycles branch-distance',
            'CNF1D:Mean of Isolated cycles mean-pixel-value',
            'CNF1D:Std of Isolated cycles mean-pixel-value',
            'CNF1D:Mean of Isolated cycles stdev-pixel-value',
            'CNF1D:Std of Isolated cycles stdev-pixel-value',
            'CNF1D:Mean of Isolated cycles euclidean-distance',
            'CNF1D:Std of Isolated cycles euclidean-distance',
            
            'CNF1D:Sholl Crossings Cyto mean',
            'CNF1D:Sholl Crossings Cyto median',
            'CNF1D:Sholl Crossings Cyto min',
            'CNF1D:Sholl Crossings Cyto max',
            'CNF1D:Sholl Crossings Cyto max_amp',
            'CNF1D:Sholl Crossings Cyto var',
            'CNF1D:Sholl Crossings Cyto std_dev',
            'CNF1D:Sholl Crossings Cyto abs_dev',
            'CNF1D:Sholl Crossings Cyto kurtosis',
            'CNF1D:Sholl Crossings Cyto skewness',
            
            'CNF1D:Sholl Crossings Nuclei mean',
            'CNF1D:Sholl Crossings Nuclei median',
            'CNF1D:Sholl Crossings Nuclei min',
            'CNF1D:Sholl Crossings Nuclei max',
            'CNF1D:Sholl Crossings Nuclei max_amp',
            'CNF1D:Sholl Crossings Nuclei var',
            'CNF1D:Sholl Crossings Nuclei std_dev',
            'CNF1D:Sholl Crossings Nuclei abs_dev',
            'CNF1D:Sholl Crossings Nuclei kurtosis',
            'CNF1D:Sholl Crossings Nuclei skewness'
    ]
}

# Create the checkboxes
checkboxes = {}
for group, features in groups.items():
    checkbox_group = []
    for feature in features:
        checkbox = widgets.Checkbox(value=True, description=feature)
        checkbox_group.append(checkbox)
    checkboxes[group] = checkbox_group

# Create the button
generate_button = widgets.Button(description='Generate List')
features = []

# Function to handle button click event
def generate_list(button):
    global features
    features = []
    for group in checkboxes.values():
        features.extend([checkbox.description for checkbox in group if checkbox.value])
    #print(f'Features: {features}')

# Attach the function to the button click event
generate_button.on_click(generate_list)

# Create the checkbox groups and add vertical spacing
groups_layout = []
for group, checkboxes_group in checkboxes.items():
    checkboxes_group_layout = widgets.HBox(checkboxes_group)
    groups_layout.append(widgets.VBox([checkboxes_group_layout, widgets.HTML('<br>')]))

# Create the final layout
layout = widgets.VBox(groups_layout + [generate_button])

# Display the layout
layout


In [7]:
import warnings
warnings.filterwarnings("ignore")

# Dropdown and Label
dropdown_image     = widgets.Dropdown(description =  'Select Image:' , options = ['Select image'] + list(DeconvDF.index))
label_text         = widgets.Text(value='', description='Label:', disabled=True)
dropdown_and_label = widgets.HBox([dropdown_image, label_text])

# Texture Sliders
# slider_gamma   = widgets.IntSlider(min=1,max=1000,step=100,description='Gamma:',value=500)
# slider_thr     = widgets.IntSlider(min=0,max=20,step=1,description='Threshold:',value=0)
# slider_linelen = widgets.FloatSlider(min=0,max=10,step=0.5,description='LineLen/Scale:',value=2.5)
# slider_linegap = widgets.IntSlider(min=0,max=10,step=1,description='LineGap:',value=1)
# sliders        = widgets.HBox([slider_gamma, slider_thr, slider_linelen, slider_linegap])

# # To Add
radiobutton_to_add = widgets.RadioButtons(options=['Yes','No'], description='Add cell to ROIs DataFrame?',value='No')

# Calculate Button
button = widgets.Button(value=False, description='Calculate', disabled=False, button_style='')

# Outputs
output_ROI      = widgets.Output()
output_RES      = widgets.Output()

# Event Handler Functions
def ROI_eventhandler(change):
    image = DeconvDF['Image'][dropdown_image.value]
    
    # Select ROI QT
    %matplotlib qt
    fig,ax = plt.subplots(figsize=(15,15))
    ax.imshow(image,cmap='gray')
    ax.axis('off')
    
    # Plot nuclei contours and centroids
    plot_nuclei_contours(CentroidsDF=Centroids,imgIndex=dropdown_image.value,ax=ax)

    # Skeleton
    fig_s,ax_s = plt.subplots(figsize=(15,15))
    ax_s.imshow(TextureDF['Skeleton'][dropdown_image.value],cmap='gray')
    ax_s.axis('off')
    
    # Plot nuclei contours and centroids
    plot_nuclei_contours(CentroidsDF=Centroids,imgIndex=dropdown_image.value,ax=ax_s)

    # Draw ROI and get Coordinates
    global ROI,mask
    ROI = RoiPoly(color='r')
    ROI.display_roi()
    mask = ROI.get_mask(image) #roi_coordinates = ROI.get_roi_coordinates()
    %matplotlib inline
    
    with output_ROI:
        plt.figure(figsize=(15,10))
        plt.subplot(1,3,1)
        
        # Plot Original
        copia = copy.deepcopy(OriginalDF['Image'][dropdown_image.value])
        auxiliar = mask * OriginalDF['Image'][dropdown_image.value][:,:,2]
        xxx,yyy = np.where(auxiliar != 0)
        helpp = np.zeros_like(auxiliar[min(xxx):max(xxx),min(yyy):max(yyy)])
        auxiliar = np.stack((auxiliar[min(xxx):max(xxx),min(yyy):max(yyy)],helpp,helpp),axis=2)
#         copia[:,:,0] = auxiliar[min(xxx):max(xxx),min(yyy):max(yyy)]
#         copia[:,:,1] = np.zeros_like(copia[:,:,0])
#         copia[:,:,2] = np.zeros_like(copia[:,:,0])
        plt.imshow(auxiliar)
        plt.title('Original')
        plt.axis('off')
        
        # Plot Deconvoluted
        plt.subplot(1,3,2)
        auxiliar = mask * image
        xxx,yyy = np.where(auxiliar != 0)
        plt.imshow(auxiliar[min(xxx):max(xxx),min(yyy):max(yyy)],cmap='gray')
        plt.title('Deconvoluted')
        plt.axis('off')
        
        # Plot Skeleton
        plt.subplot(1,3,3)
        sk = TextureDF['Skeleton'][dropdown_image.value]
        plt.imshow(np.max(sk[min(xxx):max(xxx),min(yyy):max(yyy)])-sk[min(xxx):max(xxx),min(yyy):max(yyy)],cmap='gray')
        plt.title('Skeleton')
        plt.axis('off')
        plt.show()
                
    # Define Texture and Analyse Parameters
    #display(sliders)
    display(radiobutton_to_add)
    #text_ske_eventhandler(change)
    
         
def text_ske_eventhandler(change):
    if type(dropdown_image.value) != int:
        algorithm = 'synthetic'
    else:
        algorithm = 1
    #texture,skeleton,TextureDF = img_getTexture([mask * DeconvDF['Image'][dropdown_image.value],dropdown_image.value,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=change.new, algorithm=1,TextureDF=TextureDF,to_add=radiobutton_to_add.value)
    skeleton = img_preprocessing(image=[OriginalDF['Image'][dropdown_image.value],dropdown_image.value], algorithm='original',parameters=[0.03,500],plot=False)

    with output_ROI:
        print('o')
        plt.clf()
        plt.figure(figsize=(10,10))
        
        x_x,y_y = np.where(texture != 0)
        #plt.imshow(texture[min(x_x):max(x_x),min(y_y):max(y_y)],cmap='gray')
        plt.axis('off')
        plt.imshow(DeconvDF['Image'][dropdown_image.value],cmap='gray')
        plt.show()
#         plt.subplot(1,2,2)
#         x_x,y_y = np.where(skeleton != 0)
#         plt.imshow(skeleton[min(x_x):max(x_x),min(y_y):max(y_y)],cmap='gray')
#         plt.axis('off')
#         plt.show()
            
def RES_eventhandler(change):
    output_RES.clear_output()
    #image = DeconvDF['Image'][dropdown_image.value]
    
    
    with output_RES:
        %matplotlib inline
        
#         if type(dropdown_image.value) != int:
#             texture,skeleton,TextureDF = img_getTexture([mask * DeconvDF['Image'][dropdown_image.value],dropdown_image.value,DeconvDF['Name'][dropdown_image.value],DeconvDF['Label'][dropdown_image.value]], sigmas=[0.03], gamma=slider_gamma.value, algorithm='synthetic',TextureDF=TextureDF,to_add=radiobutton_to_add.value)  
#         else:
         
        global ImageLinesDF
        %matplotlib qt
        if radiobutton_to_add.value == 'texture':
            ImageLinesDF = analyze_cell([texture, dropdown_image.value],mask,[slider_thr.value,slider_linelen.value,slider_linegap.value],Centroids[dropdown_image.value],DeconvDF,NucleiDeconvDF,True)
        if radiobutton_to_add.value == 'skeleton':
            ImageLinesDF = analyze_cell([skeleton, dropdown_image.value],mask,[slider_thr.value,slider_linelen.value,slider_linegap.value],Centroids[dropdown_image.value],DeconvDF,NucleiDeconvDF,True)
        
        #ResultsDF = analyze_cell([skeleton, dropdown_image.value],mask,[2,2.5,1],Centroids[dropdown_image.value],DeconvDF,NucleiDeconvDF,True)
        ResultsDF = analyze_cell([skeleton, dropdown_image.value,texture],row['ROImask'],[2,2.5,1],Centroids[row['Index']],OriginalDF,DeconvDF,NucleiDeconvDF,True)
        
        quantitative_analysis(ResultsDF)

def dropdown_image_eventhandler(change):
    output_ROI.clear_output()
    output_RES.clear_output()
    
    global img_id
    img_id = dropdown_image.value
    label_text.value = DeconvDF['Label'][dropdown_image.value]
    
    with output_ROI:
        ROI_eventhandler(change)
        #text_ske_eventhandler(change)
        display(button)
        
# OBSERVE    
#radiobuttons_original.observe(radiobuttons_original_eventhandler, names='value')
dropdown_image.observe(dropdown_image_eventhandler, names='value')
                             
# Display initial widgets
display(dropdown_and_label)

# CALCULATE AND ADD ROI
with output_RES:
    button.on_click(RES_eventhandler)
    if radiobutton_to_add.value == 'Yes':
        new  = pd.DataFrame(data = {'Name': [DeconvDF['Name'][img_id]],'Index': [img_id], 'Label': [DeconvDF['Label'][img_id]], 'ROImask': [mask]})
        ROIs = ROIs.append(new,ignore_index=True)

# TABS
tab = widgets.Tab([output_ROI, output_RES])
tab.set_title(0, 'Select ROI'); tab.set_title(1, 'Results')

display(tab)